In [ ]:
#!pip install CommonRegex
#!pip install pgeocode

In [9]:
import pytesseract 
from PIL import Image, ImageOps
import re
import numpy as np
from commonregex import CommonRegex
import pandas as pd
import pgeocode

# from postal.parser import parse_address

In [10]:
texts_df = pd.read_csv('texts.csv')
text = texts['0'][0]
times = CommonRegex(text).times

#pri

In [11]:

us_zip = r'(\d{5}\-?\d{0,4})'
zip_code = re.search(us_zip, text)
zip_code = zip_code[0]

nomi = pgeocode.Nominatim('us')
geodata = nomi.query_postal_code(zip_code)

In [12]:
def extract_prices(text):
    return CommonRegex(text).prices

In [13]:
def extract_geo(text):
    #zip_code 
    us_zip = r'(\d{5}\-?\d{0,4})'
    zip_code = re.search(us_zip, text)
    try:
        zip_code = zip_code[0]
        nomi = pgeocode.Nominatim('us')
        geodata = nomi.query_postal_code(zip_code)
        longitude = geodata['longitude']
        latitude = geodata['latitude']
    except:
        zip_code = np.nan
        longitude = np.nan
        latitude = np.nan
        
    
    address = CommonRegex(text).street_addresses
    if len(address) == 1:
        address =  address[0]
    else:
        address = np.nan

        
    return zip_code, longitude, latitude, address
    
    

In [14]:
def extract_time(text):
    times = CommonRegex(text).times
    if len(times) == 1:
        return times[0]
    else:
        return np.nan

In [16]:
#texts_df = pd.read_csv('texts.csv')
geos = []
times = []
for text in texts_df['0']:
    times.append(extract_time(text))
    geos.append(extract_geo(text))



In [17]:
import pandas as pd
df = pd.DataFrame(data = geos, columns = ['Zipcode','Latitude', 'Longitude', 'Address' ])
df['Time'] = times

In [32]:
prices = []
for text in texts_df['0']:
    prices.append(extract_prices(text))
df['prices'] = prices


database = df.explode('prices')
database.reset_index(inplace = True)

In [36]:
database.isna().sum()

index          0
Zipcode       83
Latitude     176
Longitude    176
Address      246
Time         143
prices       130
dtype: int64

In [37]:
df.isna().sum()

Zipcode       62
Latitude     111
Longitude    111
Address      130
Time          91
prices         0
dtype: int64